In [1]:
%load_ext autoreload
%autoreload 2
import pynwb
import os
import numpy as np

# DataJoint and DataJoint schema
import datajoint as dj

dj.config["filepath_checksum_size_limit"] = 1 * 1024**2

## We also import a bunch of tables so that we can call them easily
from spyglass.common import (
    ElectrodeGroup,
    Electrode,
    FirFilterParameters,
    IntervalList,
    Session,
    Task,
    TaskEpoch,
    Nwbfile,
    AnalysisNwbfile,
    NwbfileKachery,
    AnalysisNwbfileKachery,
    interval_list_contains,
    interval_list_contains_ind,
    interval_list_excludes,
    interval_list_excludes_ind,
    interval_list_intersect,
    get_electrode_indices,
)

from spyglass.lfp.v1 import (
    LFPElectrodeGroup,
    LFPSelection,
    LFPV1,
)
from spyglass.lfp.lfp_merge import LFPOutput


from spyglass.lfp.analysis.v1.lfp_band import LFPBandSelection,LFPBandV1
# from spyglass.lfp_band.lfp_band_merge import LFPBandOutput
import os
import warnings
from tqdm import tqdm
warnings.simplefilter("ignore", category=DeprecationWarning)
warnings.simplefilter("ignore", category=ResourceWarning)
warnings.simplefilter("ignore", category=UserWarning)

[2023-10-26 13:10:57,434][INFO]: Connecting sambray@lmf-db.cin.ucsf.edu:3306
[2023-10-26 13:10:57,483][INFO]: Connected sambray@lmf-db.cin.ucsf.edu:3306


## Define Animal Sessions

In [2]:
nwb_file_names = [
    #Olive
    "Olive20220624_.nwb",
    "Olive20220627_.nwb",
    "Olive20220628_.nwb",
    "Olive20220629_.nwb",
    "Olive20220630_.nwb",
    "Olive20220701_.nwb",
    "Olive20220702_.nwb",
    "Olive20220705_.nwb",
    "Olive20220706_.nwb",
    #Yoshi
    "Yoshi20220509_.nwb",
    "Yoshi20220510_.nwb",
    "Yoshi20220511_.nwb",
    "Yoshi20220519_.nwb",
    "Yoshi20220525_.nwb",
    #Winnie
    "Winnie20220712_.nwb",
    "Winnie20220713_.nwb",
    "Winnie20220714_.nwb",
    "Winnie20220715_.nwb",
    "Winnie20220716_.nwb",
    #Wallie
    "Wallie20220911_.nwb",
    "Wallie20220912_.nwb",
    "Wallie20220913_.nwb",
    "Wallie20220916_.nwb",
    #Totoro
    "Totoro20220524_.nwb",
    "Totoro20220525_.nwb",
    "Totoro20220526_.nwb",
    "Totoro20220527_.nwb",
    "Totoro20220530_.nwb",
    "Totoro20220531_.nwb",
    "Totoro20220602_.nwb",
    "Totoro20220603_.nwb",
    "Totoro20220606_.nwb",
    "Totoro20220621_.nwb",
    #Odins
    "Odins20220107_.nwb",
    "Odins20220110_.nwb",
    "Odins20220111_.nwb",
    "Odins20220113_.nwb",
    "Odins20220118_.nwb",
    "Odins20220119_.nwb",
    "Odins20220123_.nwb",
    "Odins20220124_.nwb",
    # "Odins20220126_.nwb", ???
    #Banner
    "Banner20211221_.nwb",
    "Banner20211222_.nwb",
    "Banner20211227_.nwb",
    "Banner20211228_.nwb",
    "Banner20211230_.nwb",
    "Banner20211231_.nwb",
    "Banner20220103_.nwb",
    "Banner20220105_.nwb",
    "Banner20220123_.nwb",
    "Banner20220124_.nwb",
    "Banner20220126_.nwb",
]

nwb_file_names.extend([
    #Olive
    "Olive20220707_.nwb",
    "Olive20220708_.nwb",
    "Olive20220711_.nwb",
    #Yoshi
    "Yoshi20220513_.nwb",
    "Yoshi20220516_.nwb",
    "Yoshi20220517_.nwb",
    "Yoshi20220518_.nwb",
    #Winnie
    "Winnie20220717_.nwb",
    "Winnie20220718_.nwb",
    "Winnie20220719_.nwb",
    "Winnie20220720_.nwb",
    "Winnie20220722_.nwb",
    #Wallie
    "Wallie20220914_.nwb",
    "Wallie20220915_.nwb",
    "Wallie20220922_.nwb",
    #Totoro
    "Totoro20220607_.nwb",
    "Totoro20220610_.nwb",
    "Totoro20220613_.nwb",
    "Totoro20220614_.nwb",
    "Totoro20220615_.nwb",
    "Totoro20220616_.nwb",
    "Totoro20220617_.nwb",
    #Odins
    "Odins20220226_.nwb",
    #Banner
    "Banner20220215_.nwb",
    "Banner20220216_.nwb",
    "Banner20220223_.nwb",
    "Banner20220224_.nwb",
    "Banner20220225_.nwb",
])
    
# np.isnan(np.mean([]))
animal_names = ["Wallie","Winnie","Olive","Totoro","Yoshi","Dan"]
animal_names = ["Bilbo","Winnie","Frodo"]
from spyglass.common import Session
nwb_file_names = []
[nwb_file_names.extend((Session() & {"subject_id":animal}).fetch("nwb_file_name")) for animal in animal_names]

nwb_file_names = ['Winnie20220714_.nwb']

#write permission?
analysis_path = "/stelmo/nwb/analysis/"
for nwb_file in nwb_file_names:
    if not os.path.isdir(analysis_path+nwb_file.split('_')[0]):
        print(f'No analysis yet for: {nwb_file}')
    elif not os.access(analysis_path+nwb_file.split('_')[0],os.W_OK):
        print(f'No write permissions for: {nwb_file}nwb_file')


## Populate

In [3]:
# nwb_file_names = ["Totoro20220602_.nwb",]
# interval_list_name = "pos 3 valid times"
filter_name = "Theta 5-11 Hz"
# desired sampling rate
lfp_band_sampling_rate = 100
missing_lfp = []



############################################
for nwb_file_name in nwb_file_names:
    for interval_list_name in (IntervalList()& {"nwb_file_name":nwb_file_name}).fetch('interval_list_name'): #["pos 7 valid times"]:#
        if ('pos' == interval_list_name[:3]) and ("valid times" == interval_list_name[-11:]):
            
            basic_key = {"nwb_file_name": nwb_file_name,'target_interval_list_name':interval_list_name}
            #check that lfp exists for this animal
            if len(LFPV1() & basic_key)==0:
                missing_lfp.append(basic_key)
                continue
            if len(LFPV1 & basic_key)>1:
                electrode_group_names = (LFPV1() & basic_key).fetch("lfp_electrode_group_name")
                electrode_group_name =[x for x in electrode_group_names if nwb_file_name.split('202')[0] in x][0]
                basic_key["lfp_electrode_group_name"] = electrode_group_name
                
            #get lfp's sampling rate
            lfp_sampling_rate = (LFPV1() & basic_key ).fetch1(
                "lfp_sampling_rate"
            )
            
            #get lfp filter sampling_rate
            lfp_filter_sampling_rate = (LFPV1() & basic_key).fetch1(
                "filter_sampling_rate"
            )
            
            #get lfp filter name
            lfp_filter_name = (LFPV1() & basic_key).fetch1(
                "filter_name"
            )
            
            # make lfp selection key
            lfp_s_key = basic_key.copy()
            lfp_s_key["target_interval_list_name"] = interval_list_name
            lfp_s_key["filter_name"] = lfp_filter_name
            lfp_s_key["filter_sampling_rate"] = lfp_filter_sampling_rate
            
            # get the electrodes the lfp was run on
            lfp_electrode_group_name = (LFPV1 & lfp_s_key).fetch1('lfp_electrode_group_name')
            lfp_eg_key = {
                    "nwb_file_name": nwb_file_name,
                    "lfp_electrode_group_name": lfp_electrode_group_name,
                    }
            electrode_list = list(np.asarray((LFPV1 & basic_key).fetch_nwb()[0]['lfp'].electrodes.to_dataframe().index))
            
            # remake lfp selection key
            lfp_s_key = basic_key.copy()
            lfp_s_key["target_interval_list_name"] = interval_list_name
            lfp_s_key["filter_name"] = lfp_filter_name
            lfp_s_key["filter_sampling_rate"] = lfp_filter_sampling_rate
            
            # set the reference to -1 to indicate no reference for all channels
            ref_elect = [-1]
            
            # we also need the uuid for the LFP object
            lfp_merge_id = (LFPOutput.LFPV1 & lfp_s_key).fetch1('merge_id')
            
            
            # #make Band selection table entry
            LFPBandSelection().set_lfp_band_electrodes(
                    nwb_file_name=nwb_file_name,
                    lfp_merge_id=lfp_merge_id,
                    electrode_list=electrode_list,
                    filter_name=filter_name,
                    interval_list_name=interval_list_name,
                    reference_electrode_list=ref_elect,
                    lfp_band_sampling_rate=lfp_band_sampling_rate,
                )
            
            lfp_band_key = (
                LFPBandSelection
                & {
                    "lfp_merge_id": lfp_merge_id,
                    "filter_name": filter_name,
                    "lfp_band_sampling_rate": lfp_band_sampling_rate,
                    "target_interval_list_name":interval_list_name,
                }
            ).fetch1("KEY")
            if LFPBandV1() & lfp_band_key:
                continue #already populated
            # lfp_band_key['lfp_electrode_group_name']="full_probe"
            LFPBandV1().populate(LFPBandSelection()&lfp_band_key)

[2023-10-26 13:11:58,353][WARNING]: Skipped checksum for file with hash: 8c73b162-70f2-aecb-1425-2e1f83e2a1c6, and path: /stelmo/nwb/analysis/Winnie20220714/Winnie20220714_ISXEFBV66J.nwb
[2023-10-26 13:11:59,362][WARNING]: Skipped checksum for file with hash: 8c73b162-70f2-aecb-1425-2e1f83e2a1c6, and path: /stelmo/nwb/analysis/Winnie20220714/Winnie20220714_ISXEFBV66J.nwb


Writing new NWB file Winnie20220714_7LE95FJS9X.nwb


[2023-10-26 13:12:23,874][WARNING]: Skipped checksum for file with hash: 58683a68-d1e8-9038-7c1c-0628a03129d4, and path: /stelmo/nwb/analysis/Winnie20220714/Winnie20220714_RKYT1QGOV3.nwb
[2023-10-26 13:12:24,713][WARNING]: Skipped checksum for file with hash: 58683a68-d1e8-9038-7c1c-0628a03129d4, and path: /stelmo/nwb/analysis/Winnie20220714/Winnie20220714_RKYT1QGOV3.nwb


Writing new NWB file Winnie20220714_VB30QZNJDR.nwb


[2023-10-26 13:12:34,037][WARNING]: Skipped checksum for file with hash: 5ece8129-70a7-a01d-9af8-fe9a1dc89f8e, and path: /stelmo/nwb/analysis/Winnie20220714/Winnie20220714_LWTP99D94J.nwb
[2023-10-26 13:12:34,728][WARNING]: Skipped checksum for file with hash: 5ece8129-70a7-a01d-9af8-fe9a1dc89f8e, and path: /stelmo/nwb/analysis/Winnie20220714/Winnie20220714_LWTP99D94J.nwb


...

Writing new NWB file Winnie20220714_GF68XT1DQ0.nwb


[2023-10-26 13:14:48,023][WARNING]: Skipped checksum for file with hash: a62d5fcb-4edc-58a4-9c92-d5c517e2ac6c, and path: /stelmo/nwb/analysis/Winnie20220714/Winnie20220714_YN6HXN35Y3.nwb
[2023-10-26 13:14:49,276][WARNING]: Skipped checksum for file with hash: a62d5fcb-4edc-58a4-9c92-d5c517e2ac6c, and path: /stelmo/nwb/analysis/Winnie20220714/Winnie20220714_YN6HXN35Y3.nwb


Writing new NWB file Winnie20220714_VG4L8W3HRE.nwb


[2023-10-26 13:15:29,533][WARNING]: Skipped checksum for file with hash: d4b3e20d-61b4-6786-a0f3-5054422aa702, and path: /stelmo/nwb/analysis/Winnie20220714/Winnie20220714_8S8864K2DZ.nwb
[2023-10-26 13:15:30,950][WARNING]: Skipped checksum for file with hash: d4b3e20d-61b4-6786-a0f3-5054422aa702, and path: /stelmo/nwb/analysis/Winnie20220714/Winnie20220714_8S8864K2DZ.nwb


Writing new NWB file Winnie20220714_CO26H9H3PZ.nwb


In [10]:
lfp_band_key

(LFPV1 & basic_key).fetch_nwb()[0]['lfp']

[2023-10-20 08:18:10,755][WARNING]: Skipped checksum for file with hash: 1f41ed1f-df73-533d-6c95-5cb459607516, and path: /stelmo/nwb/analysis/Bilbo20230725/Bilbo20230725_JRH9I6411O.nwb


filtered data pynwb.ecephys.ElectricalSeries at 0x140401433184672
Fields:
  comments: no comments
  conversion: 1.0
  data: <HDF5 dataset "data": shape (2016507, 14), type "<i2">
  description: filtered data
  electrodes: electrodes <class 'hdmf.common.table.DynamicTableRegion'>
  interval: 1
  offset: 0.0
  resolution: -1.0
  timestamps: <HDF5 dataset "timestamps": shape (2016507,), type "<f8">
  timestamps_unit: seconds
  unit: volts

## Check missing LFP's

In [13]:
print(missing_lfp)

'Banner20220225_.nwb'